# Navigation of the robot


In [ ]:
# Let's intitialize the variables

speed0 = 100       #initial speed
speed_gain = 2     # 
obstThrL = 10      # Low threshold for which the robot exits the avoidance state
obstThrH = 20      # High threshold for which the robot enters the avoidance state
obstSpeedGain = 5  # gain associated to obstacle avoidance 

state = 1         
prox = [0,0,0,0,0,0,0] # obstacle detection array


